In [175]:
import numpy as np
import pandas as pd
from datetime import datetime

In [176]:
rides_df = pd.read_csv('cab_rides.csv')
weather_df = pd.read_csv('weather.csv')

### Merging the two csv-files

In [177]:
# Converting the timestamp data into real date format
rides_df['date_time'] = pd.to_datetime(rides_df['time_stamp']/1000, unit='s')
weather_df['date_time'] = pd.to_datetime(weather_df['time_stamp'], unit='s')

In [178]:
# Merge the datasets to reflect the same time for a location, date and hour
rides_df['merged_date'] = rides_df.source.astype(str) +" - "+ rides_df.date_time.dt.date.astype("str") + " - "+ rides_df.date_time.dt.hour.astype("str")
weather_df['merged_date'] = weather_df.location.astype(str) +" - "+ weather_df.date_time.dt.date.astype("str") + " - "+ weather_df.date_time.dt.hour.astype("str")

In [179]:
merged_df = pd.merge(rides_df, weather_df, on=['merged_date'])

# Merge from Kaggle, but i used the one above becuase it seems more intuitive 
#merged_df = UberLyft_df.join(Weather_df, on = ['merged_date'], rsuffix ='_w')

In [180]:
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1265675 entries, 0 to 1265674
Data columns (total 21 columns):
distance            1265675 non-null float64
cab_type            1265675 non-null object
time_stamp_x        1265675 non-null int64
destination         1265675 non-null object
source              1265675 non-null object
price               1164996 non-null float64
surge_multiplier    1265675 non-null float64
id                  1265675 non-null object
product_id          1265675 non-null object
name                1265675 non-null object
date_time_x         1265675 non-null datetime64[ns]
merged_date         1265675 non-null object
temp                1265675 non-null float64
location            1265675 non-null object
clouds              1265675 non-null float64
pressure            1265675 non-null float64
rain                206947 non-null float64
time_stamp_y        1265675 non-null int64
humidity            1265675 non-null float64
wind                1265675 non-null f

As we can see the merged dataframe consists of 1.265.675 entries while the original dataset had 693.071 entries this is because the weather data has been recorded more than one time for the same hour, and since we merge by hour (since the rides and weather do not have the exact same timestamp), then we end up with rides having different weather attributes adn therefore more the one row of ride than one. This can be seen in the following since each id can occur multiple times.

### Showing an fixing the merge issue explained above

In [181]:
# Count of how many instances there is of each trip
merged_df['id'].value_counts()

1b446c5a-efd1-4004-9275-13bb456af595    15
5f66a01e-d637-4c8a-9136-5c2fee0c1992    15
d035a22f-c58d-4469-9504-382ae1d2f7c6    15
f34605f4-e1cc-4217-88a0-44c37afc36c2    15
72db179b-ef0b-44e6-8d26-6ea60e5f6bfa    15
                                        ..
07ea06ca-8a1d-42f8-94df-d1e79a4815de     1
f859c81b-618e-4563-bbbc-2fe3e3bbe2eb     1
bbb7589e-98b3-49ff-ab3f-debd1aa955b3     1
e0c2b7cb-4874-4b50-b6a5-72d099434649     1
e84c3b4a-5293-449a-aa26-5f6a83266cff     1
Name: id, Length: 690107, dtype: int64

In [182]:
# An example of a specific id that occurs multiple times
merged_df[merged_df['id'] == '29ce4d5f-07f9-4ce5-ada4-ae9ea49e92a6'].iloc[:,10:22]

,date_time_x,merged_date,temp,location,clouds,pressure,rain,time_stamp_y,humidity,wind,date_time_y
900564,2018-11-26 06:14:16.085999966,North End - 2018-11-26 - 6,40.54,North End,1.00,1014.19,NaN,1543212884,0.91,1.25,2018-11-26 06:14:44
900565,2018-11-26 06:14:16.085999966,North End - 2018-11-26 - 6,40.55,North End,1.00,1014.19,NaN,1543213005,0.91,1.28,2018-11-26 06:16:45
900566,2018-11-26 06:14:16.085999966,North End - 2018-11-26 - 6,40.61,North End,1.00,1014.18,NaN,1543213443,0.91,1.38,2018-11-26 06:24:03
900567,2018-11-26 06:14:16.085999966,North End - 2018-11-26 - 6,40.53,North End,1.00,1014.19,NaN,1543212732,0.91,1.22,2018-11-26 06:12:12
900568,2018-11-26 06:14:16.085999966,North End - 2018-11-26 - 6,40.55,North End,1.00,1014.19,NaN,1543212945,0.91,1.27,2018-11-26 06:15:45
900569,2018-11-26 06:14:16.085999966,North End - 2018-11-26 - 6,40.55,North End,1.00,1014.19,NaN,1543212914,0.91,1.26,2018-11-26 06:15:14
900570,2018-11-26 06:14:16.085999966,North End - 2018-11-26 - 6,40.53,North End,1.00,1014.19,NaN,1543212708,0.91,1.21,2018-11-26 06:11:48
900571,2018-11-26 06:14:16.085999966,North End - 2018-11-26 - 6,41.16,North End,0.98,1014.35,NaN,1543215030,0.92,1.83,2018-11-26 06:50:30
900572,2018-11-26 06:14:16.085999966,North End - 2018-11-26 - 6,40.64,North End,1.00,1014.18,NaN,1543213742,0.92,1.45,2018-11-26 06:29:02
900573,2018-11-26 06:14:16.085999966,North End - 2018-11-26 - 6,40.54,North End,1.00,1014.19,NaN,1543212855,0.91,1.25,2018-11-26 06:14:15


In [183]:
# since the weather data for each id is very similar, we choose to make a mean of all the instances by id
id_group = pd.DataFrame(merged_df.groupby('id')['temp','clouds', 'pressure', 'rain', 'humidity', 'wind'].mean())
merged_uniqueid_df = rides_df.join(id_group, on = ['id'])

In [184]:
merged_uniqueid_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 693071 entries, 0 to 693070
Data columns (total 18 columns):
distance            693071 non-null float64
cab_type            693071 non-null object
time_stamp          693071 non-null int64
destination         693071 non-null object
source              693071 non-null object
price               637976 non-null float64
surge_multiplier    693071 non-null float64
id                  693071 non-null object
product_id          693071 non-null object
name                693071 non-null object
date_time           693071 non-null datetime64[ns]
merged_date         693071 non-null object
temp                690107 non-null float64
clouds              690107 non-null float64
pressure            690107 non-null float64
rain                118677 non-null float64
humidity            690107 non-null float64
wind                690107 non-null float64
dtypes: datetime64[ns](1), float64(9), int64(1), object(7)
memory usage: 95.2+ MB


In [185]:
merged_uniqueid_df['id'].value_counts()

d0048593-9b86-4c58-bfa6-0dac473846ba    1
3c71fad4-08c5-451d-816a-7336a07f33b3    1
b6dea706-1d2d-48f9-ae4d-8615ecd385f3    1
0f49c48a-b83c-4bd9-99a5-60338118b8fb    1
8cad2d6d-e04c-49b7-8d57-97536bded006    1
                                       ..
c113c9df-3420-4d2f-89d2-48d0d3938ec4    1
248172c1-a34b-4a44-ace8-1029919d7b69    1
8f9d2f94-f066-42f2-9f4e-0b4c98c483c9    1
f38fa6b2-f45d-4860-a1af-f945564e8e51    1
2ba70250-367f-4d0b-a1c1-de022185f165    1
Name: id, Length: 693071, dtype: int64

In [186]:
# Renaming the df to just "df" since it is our final merged df
df = merged_uniqueid_df

### Cleaning the data

#### Null values

In [187]:
# Checking for null values
df.isnull().sum()

distance                 0
cab_type                 0
time_stamp               0
destination              0
source                   0
price                55095
surge_multiplier         0
id                       0
product_id               0
name                     0
date_time                0
merged_date              0
temp                  2964
clouds                2964
pressure              2964
rain                574394
humidity              2964
wind                  2964
dtype: int64

In [188]:
# We assume the null values for 'rain' mean that it did not rain, therefore we change these values to '0'
df['rain'].fillna(0,inplace=True)

In [189]:
# Dropping all instances where the price and some wheather conditions is null, assuming this is a mistake
df.dropna(inplace=True)

#### Attribute cleaning

In [190]:
"""
Removing attributes we do not need. This includes date_time since it is incorrect, but we will create a new
attribute of this later based on 'time_stamp'. It also involves 'id' since we can just use the index.
Also rearraging the attributes in an order that makes more sense.
"""
df = df[['cab_type', 'distance', 'time_stamp', 'source', 'destination', 'price', 'surge_multiplier',
         'name', 'temp', 'clouds', 'pressure', 'rain', 'humidity', 'wind']]

In [191]:
# Resetting the index since we have removed several rows
df.reset_index(drop=True, inplace=True)

### Adding attributes

In [192]:
# Adding a correct 'date' and 'time' based on 'time_stamp'
df["rounded_timestamp"] = df["time_stamp"] / 1000
df["rounded_timestamp"] = df["rounded_timestamp"].apply(np.floor)

df["date"] = df["rounded_timestamp"].apply(lambda x : datetime.fromtimestamp(x).date())
df["time"] = df["rounded_timestamp"].apply(lambda x: datetime.fromtimestamp(x).time())

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = val

In [193]:
# Creating new attributes for weekday, hour of the day and time of the day (if it is morning, afternoon, evening or night)
df['weekday'] = df['date'].apply(lambda x: x.weekday())
df["weekday"] = df["weekday"].map({0: 'Monday', 1: 'Tuesday', 2: 'Wednesday', 3: 'Thursday', 4: 'Friday', 5: 'Saturday', 6: 'Sunday'})
df['hour'] = df['time'].apply(lambda time: time.hour)

df.loc[(df.hour >= 6) & (df.hour < 12) , 'time_of_day'] = 'Morning'
df.loc[(df.hour >= 12) & (df.hour < 15) , 'time_of_day'] = 'Afternoon'
df.loc[(df.hour >= 15) & (df.hour < 18) , 'time_of_day'] = 'Evening'
df.loc[(df.hour >= 18) | (df.hour < 6) , 'time_of_day'] = 'Night'

In [194]:
# Changing the date to a datetime datetype instead of object
df['date'] = pd.to_datetime(df['date'])

"""
Can also be changed to a simple Integer if needed
df["date"] = pd.to_datetime(df["date"]).dt.strftime("%Y%m%d")
df['date'].astype(int)
"""

'\nCan also be changed to a simple Integer if needed\ndf["date"] = pd.to_datetime(df["date"]).dt.strftime("%Y%m%d")\ndf[\'date\'].astype(int)\n'

In [195]:
# Rearranging the new attributes and removing 'time_stamp' and 'time' since we do not need it anymore
df = df[['cab_type', 'distance', 'source', 'destination', 'price', 'surge_multiplier', 'name',
         'date', 'weekday', 'hour', 'time_of_day',
         'temp', 'clouds', 'pressure', 'rain', 'humidity', 'wind']]

In [196]:
df.head()

,cab_type,distance,source,destination,price,surge_multiplier,name,date,weekday,hour,time_of_day,temp,clouds,pressure,rain,humidity,wind
0,Lyft,0.44,Haymarket Square,North Station,5.0,1.0,Shared,2018-12-16,Sunday,10,Morning,38.460,0.290000,1022.25,0.000,0.760000,7.68
1,Lyft,0.44,Haymarket Square,North Station,11.0,1.0,Lux,2018-11-27,Tuesday,3,Night,44.065,0.995000,1002.88,0.106,0.895000,12.63
2,Lyft,0.44,Haymarket Square,North Station,26.0,1.0,Lux Black XL,2018-11-30,Friday,5,Night,35.080,0.000000,1013.71,0.000,0.700000,5.25
3,Lyft,0.44,Haymarket Square,North Station,9.0,1.0,Lyft XL,2018-11-29,Thursday,4,Night,37.680,0.433333,998.42,0.000,0.706667,11.16
4,Lyft,0.44,Haymarket Square,North Station,16.5,1.0,Lux Black,2018-12-17,Monday,19,Night,40.780,0.930000,1000.15,0.000,0.790000,7.55


### Creating new dataframes

In [197]:
"""
Making two dataframes based on 'cab_type'. By this creating one for Uber and one for
Lyft in order to analyze them seperately and compare them.
"""
uber_df = df[df['cab_type'] =="Uber"]
lyft_df = df[df['cab_type'] =="Lyft"]

"""
Status:
We now have 3 dataframes: 'df', 'uber_df', 'lyft_df'. These are merged with the weather based
on the source location, the date and the hour of the date.

To do in date prep:
- Fix headlines -> Morten
- Fix headlines -> Morten
- Make baseline and importances -> Morten
- Make visualization -> see Data prep_cab or https://www.kaggle.com/cempek/uber-vs-lyft-visualizations from step 11

- Remove 'hour' attribute if we do not use it
- Change datetypes if we can't use them ('datetime' and object datetypes e.g., 'cab_type' and 'source'
    - Syntax for object to int for binary values -> df['cab_type'] = pd.get_dummies(df['cab_type'])

To do under modelling:
- Clean 'surge_multiplier' if needed in the model
- Normalize attributes to make them more hetereogenous if needed in the model
"""

In [198]:
df.to_csv(r'PreppedDataSet.csv', index=False)
uber_df.to_csv(r'UberDataSet.csv', index=False)
lyft_df.to_csv(r'LyftDataSet.csv', index=False)